# The Battle of Neighborhoods - Week 4 - Part 2

## Data
As taught in this course the Foursquare API will be used to retrieve data about nearby locations, reviews, visitors, and others. In addition to Foursquare API, data from http://cep.la/baixar also will be utulized, this data represents the postal code of the state os São Paulo with its sity, neighbourhood and street, link to the data http://cep.la/CEP-dados-2018-UTF8.zip.
Furthermore, this data gathered from Fourquare will be used to explore the locations, find and describe venues and also feature engineering. This processes will be necessary to perform the clustering (K-Means), with will determine the similarity between the neighborhoods and group them in clusters in order to identify them better.

### Loading Libraries and Data Gathering

In [2]:
import pandas as pd
import numpy as np
import zipfile
import csv
print('Libraries imported.')

Libraries imported.


In [3]:
!wget -q -O 'CEP-dados-2018-UTF8.zip' http://cep.la/CEP-dados-2018-UTF8.zip
print('Data downloaded!')

Data downloaded!


In [4]:
with zipfile.ZipFile("CEP-dados-2018-UTF8.zip","r") as zip_ref:
    zip_ref.extractall("./")
print('Data unziped!')

Data unziped!


In [5]:
df = pd.read_fwf('ceps.txt', header = None)
df.head()

,0,1,2,3,4,5,6
0,1001000,São,Paulo/SP,Sé\tPraça da Sé - lado ímpar,NaN,NaN,NaN
1,1001001,São,Paulo/SP,Sé\tPraça da Sé - lado par,NaN,NaN,NaN
2,1001010,São,Paulo/SP,Sé\tRua Filipe de Oliveira,NaN,NaN,NaN
3,1001900,São,Paulo/SP,"Sé\tPraça da Sé, 108 \t UNESP - Universidade E...",NaN,NaN,NaN
4,1001901,São,Paulo/SP,"Sé\tPraça da Sé, 371 \t Edifício Santa Lídia",NaN,NaN,NaN


### Data Cleansing and Feature Engineering

In [6]:
df1 = df[1] + " " + df[2]
df2 = pd.DataFrame(df1, columns=["CS"])


df2 = df2["CS"].str.split('/', expand=True)
df2.columns = ['City', 'State']
df2['CEP'] = df[0]

df2.head()

,City,State,CEP
0,São Paulo,SP,1001000
1,São Paulo,SP,1001001
2,São Paulo,SP,1001010
3,São Paulo,SP,1001900
4,São Paulo,SP,1001901


In [7]:
df3 = pd.DataFrame(df[3])

df4 = df3[3].str.split('\t', expand=True)
df4['CEP'] = df[0]

df5 = df4.drop([2, 3], axis = 1)
df5.columns = ['Neighborhood', 'Street', 'CEP']

df5.head()

,Neighborhood,Street,CEP
0,Sé,Praça da Sé - lado ímpar,1001000
1,Sé,Praça da Sé - lado par,1001001
2,Sé,Rua Filipe de Oliveira,1001010
3,Sé,"Praça da Sé, 108",1001900
4,Sé,"Praça da Sé, 371",1001901


In [8]:
dfm = pd.merge(df2, df5, on='CEP')
dfm.head()

,City,State,CEP,Neighborhood,Street
0,São Paulo,SP,1001000,Sé,Praça da Sé - lado ímpar
1,São Paulo,SP,1001001,Sé,Praça da Sé - lado par
2,São Paulo,SP,1001010,Sé,Rua Filipe de Oliveira
3,São Paulo,SP,1001900,Sé,"Praça da Sé, 108"
4,São Paulo,SP,1001901,Sé,"Praça da Sé, 371"


In [9]:
dfm.tail()

,City,State,CEP,Neighborhood,Street
732758,São João Bos,None,99975970,o (Ciríaco)/RS - Distrito,Centro
732759,Cru altinha,None,99978000,Ciríaco)/RS - Distrito,None
732760,Dav d Canaba,None,99980000,ro/RS,None
732761,Dav d Canaba,None,99980970,ro/RS,Centro
732762,Dav d Canaba,None,99980974,ro/RS,Capela São José do Capingui (Distrito)


In [23]:
dt = dfm[dfm['City'] == 'São Paulo']
dt.tail()

,City,State,CEP,Neighborhood,Street
88115,São Paulo,SP,8490885,Jardim Pedra Branca,Rua Cambará-Branco
88116,São Paulo,SP,8490886,Jardim Pedra Branca,Rua Erva-Baleeira
88117,São Paulo,SP,8490890,Jardim Pedra Branca,Rua Buritizinho
88118,São Paulo,SP,8490895,Jardim Pedra Branca,Rua Rosa da Venezuela
88119,São Paulo,SP,8491030,Guaianazes,Rua Lauro


#### Dropping last three digits of CEP to enlarge the grain od the postal code and get more meaningful data from foursquare

In [24]:
dt['CEPred'] = dt['CEP'].astype(str).str[:-3].astype(np.int64)
dt.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,City,State,CEP,Neighborhood,Street,CEPred
0,São Paulo,SP,1001000,Sé,Praça da Sé - lado ímpar,1001
1,São Paulo,SP,1001001,Sé,Praça da Sé - lado par,1001
2,São Paulo,SP,1001010,Sé,Rua Filipe de Oliveira,1001
3,São Paulo,SP,1001900,Sé,"Praça da Sé, 108",1001
4,São Paulo,SP,1001901,Sé,"Praça da Sé, 371",1001
